<a href="https://colab.research.google.com/github/tikendraw/chatbot-with-attention/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot Training



In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import sys
import os

if 'google.colab' in sys.modules:
    
    # Mount Google drive
    # from google.colab import drive
    # drive.mount('/content/drive')
    
    ! git clone https://github.com/tikendraw/chatbot-with-attention.git 
    os.chdir('chatbot-with-attention') 
    print(os.getcwd())

    ! pip install tensorflow==2.11 -q
    ! pip install tensorflow-text -q


Cloning into 'chatbot-with-attention'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 240 (delta 20), reused 51 (delta 14), pack-reused 176
Receiving objects: 100% (240/240), 220.49 MiB | 17.53 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Updating files: 100% (36/36), done.
/content/chatbot-with-attention/chatbot-with-attention


In [39]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    TextVectorization, 
    Embedding, 
    LSTM, 
    GRU, 
    Bidirectional, 
    TimeDistributed, 
    Dense, 
    Attention, 
    MultiHeadAttention,
    Concatenate
)
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_text as tf_text
import pickle
from datetime import datetime
from tensorflow.keras.callbacks import CSVLogger

print('GPU Avaliable: ', gpu:=len(tf.config.list_physical_devices('GPU')))

GPU Avaliable:  1


# Hyperparameters

In [40]:
MAX_OUTPUT_LENGTH = 200
BATCH_SIZE = 32
UNITS = 64
EMBEDDING_DIMS = 50

# Vectorizer

In [41]:
# preprocessing text
def tf_lower_and_split_punct_en(text):
    # Split accented characters.
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿|]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text



In [42]:
# Loading vectorizer
from_disk = pickle.load(open("./components/vectorizer.pkl", "rb"))
vectorizer = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorizer.set_weights(from_disk['weights'])

# Lets see the Vector for word "this"
# print (vectorizer("who am i"))

# Embedding

In [43]:
import numpy as np
import zipfile


In [44]:
!ls

 components			    LICENSE
'cornell movie-dialogs corpus'	    movie_dialogue_chatbot.ipynb
 cornell_movie_dialogs_corpus.zip   preprocessing_cornelldata.py
 dataset			    preprocess.py
 dataset.csv			    README.md
 embedding			    TextFile.txt
 funcyou			    training.ipynb


In [45]:

zipf = zipfile.ZipFile('./embedding/embedding_matrix.zip')
zipf.extractall('./embedding/')
zipf.close()

In [46]:
# Loading embedding_matrix
embedding_matrix = np.load('./embedding/embedding_matrix.npy')

In [47]:
embedding_matrix.shape

(59905, 50)

# Dataset

In [48]:
save_train_data_path = './dataset/train/'
save_test_data_path = './dataset/test/'

#loading the data
train_data = tf.data.Dataset.load(save_train_data_path, compression='GZIP')
test_data = tf.data.Dataset.load(save_test_data_path, compression='GZIP')

In [49]:
for (enc_input, dec_input), dec_output  in train_data.take(1):
    print('encoder input')
    print(enc_input[0, :20].numpy())
    print('-'*44)
    print('decoder input')
    print(dec_input[0, :20].numpy()) 
    print('-'*44)
    print('encoder output')
    print(dec_output[0, :20].numpy())
    break

encoder input
[    3    58    33   115    22   993     7 48680 52632    13  4132  8573
    40   421    82  3902 24129   978   516    54]
--------------------------------------------
decoder input
[    3    58    33   115    22   993     7 48680 52632    13  4132  8573
    40   421    82  3902 24129   978   516    54]
--------------------------------------------
encoder output
[   58    33   115    22   993     7 48680 52632    13  4132  8573    40
   421    82  3902 24129   978   516    54    36]


# Attention

In [50]:
class AttentionLayer(tf.keras.layers.Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


# Encoder

In [51]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, text_vectorizer, units, embed_dims, embedding_matrix=None):
        super(Encoder, self).__init__()
        self.text_vectorizer =  text_vectorizer
        self.units = units
        self.embed_dims = embed_dims
        self.vocab_size = text_vectorizer.vocabulary_size()
        self.embedding = Embedding(input_dim=self.vocab_size , output_dim=self.embed_dims, mask_zero=True, trainable=False)
        self.embedding.build((None,))
        self.embedding.set_weights([embedding_matrix])
        self.rnn = Bidirectional(merge_mode='concat', layer = LSTM(self.units, return_sequences=True, return_state=True))
        
    def call(self, x, y=None, return_state=False):
        
        x = self.embedding(x)
        encoder_output, forward_h, forward_c, backward_h, backward_c, = self.rnn(x)
        
        state_h = Concatenate()([forward_h, backward_h])
        state_c = Concatenate()([forward_c, backward_c])
        
        encoder_state = [state_h, state_c]
        
        if return_state:
            return encoder_output, encoder_state
        else:
            return encoder_output
        
    def convert_input(self, texts, return_state=False):
        texts = tf.convert_to_tensor(texts)
        if len(texts.shape) == 0:
            texts = tf.convert_to_tensor(texts)[tf.newaxis]
        context = self.text_vectorizer(texts)
        
        context = self(context, return_state = return_state)
        
        return context

# Decoder

In [52]:
class Decoder(keras.layers.Layer):
    def __init__(self, text_vectorizer, units,  embed_dims, embedding_matrix=None) :
        super(Decoder, self).__init__()
        self.text_vectorizer =  text_vectorizer
        self.units = units * 2
        self.embed_dims = embed_dims
        self.vocab_size = text_vectorizer.vocabulary_size()
        
        self.embedding = Embedding(input_dim=self.vocab_size , output_dim=self.embed_dims, mask_zero=True, trainable=False)
        self.embedding.build((None,))
        self.embedding.set_weights([embedding_matrix])
        
        self.rnn = LSTM(self.units, return_sequences=True, return_state=True)
        
        # self.attention =  tf.keras.layers.Attention()
        self.attention = AttentionLayer()
        
        self.output_dense = Dense(self.vocab_size)
        
        self.word_to_id = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)
        
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')

    def call(self, x, context, state=None, return_state = False, training=False):
        ''' x, context, state=None, return_sequence=False '''
        
        x = self.embedding(x)
        
        decoder_output, decoder_state_h, decoder_state_c = self.rnn(x, initial_state=state)
        decoder_state = [decoder_state_h, decoder_state_c]
        
        # attention
        # attn_op= self.attention([context, decoder_output])
        attn_op, attn_state = self.attention([context, decoder_output]) # this is for custom AttentionLayer()

        x = Concatenate(axis=-1)([decoder_output, attn_op])
        # x = tf.multiply(decoder_output, attn_op)
        # x = self.attention(decoder_output, context)
        # self.last_attention_weights = self.attention.last_attention_weights

        logits = self.output_dense(x)
            
        if return_state:
            return logits, decoder_state
        else:
            return logits
        
    def get_initial_state(self, context):
        batch_size = tf.shape(context)[0]
        start_tokens = tf.fill([batch_size, 1], self.start_token)
        done = tf.zeros([batch_size, 1], dtype=tf.bool)
        embedded = self.embedding(start_tokens)
        return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

    
    def tokens_to_text(self, tokens):
        words = self.id_to_word(tokens)
        result = tf.strings.reduce_join(words, axis=-1, separator=' ')
        result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
        result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
        return result
    
    def get_next_token(self, next_token, context,  done, state, temperature = 0.0):
        
        logits, state = self(next_token, context, state = state, return_state=True, training = False) 

        if temperature == 0.0:
            next_token = tf.argmax(logits, axis=-1)
        else:
            logits = logits[:, -1, :]/temperature
            next_token = tf.random.categorical(logits, num_samples=1)

        # If a sequence produces an `end_token`, set it `done`
        done = done | (next_token == self.end_token)
        # Once a sequence is done it only produces 0-padding.
        next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

        return next_token, done, state

In [53]:
# Build the encoder and decoder
encoder = Encoder(vectorizer, UNITS, EMBEDDING_DIMS, embedding_matrix)
decoder = Decoder(vectorizer, UNITS, EMBEDDING_DIMS, embedding_matrix)
        
context, x = enc_input, dec_input
context = encoder(context)
logits = decoder(x, context, training = False)

print(len(logits))

# decoder_output, att_op = logits[1]
# print(decoder_output.shape, att_op.shape)

32


In [54]:
context.shape, logits.shape

(TensorShape([32, 199, 128]), TensorShape([32, 199, 59905]))

In [55]:
# plt.figure(figsize = (30, 10))
# p1 = decoder_output[0,:,:]
# p2 = att_op[0,:,:]

# plt.subplot(1,3,1)
# sns.heatmap(p1)


# plt.subplot(1,3,2)
# sns.heatmap(p2)


# plt.subplot(1,3,3)
# sns.heatmap(tf.multiply(p2,p1))

# Model

In [56]:
class ChatBot(tf.keras.Model):
    
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, text_processor, units, embed_dims):
        super().__init__()
        self.text_processor = text_processor
        self.units = units
        self.embed_dims = embed_dims
        
        # Build the encoder and decoder
        encoder = Encoder(text_processor, units, embed_dims, embedding_matrix)
        decoder = Decoder(text_processor, units, embed_dims, embedding_matrix)
        
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        context, x = inputs
        context = self.encoder(context)
        logits = self.decoder(x, context, training = True)

        #TODO(b/250038731): remove this
        try:
          # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
            del logits._keras_mask
        except AttributeError:
            pass

        return logits


In [57]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [58]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)

# Compile and train

In [59]:
model = ChatBot(vectorizer, UNITS, EMBEDDING_DIMS)

In [60]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005),
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [61]:
EPOCHS = 10

CKPT_DIR = './model_checkpoint'
# CKPT_DIR = '/content/drive/MyDrive/tf_model/chatbot'
os.makedirs(CKPT_DIR, exist_ok = True)
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(CKPT_DIR,  f"{datetime.now().strftime('%m:%d:%Y, %H:%M:%S')}"),
    monitor= 'masked_acc',
    verbose= 0,
    save_best_only = True,
    save_weights_only = True,
    mode= 'auto',
    save_freq='epoch'
)

os.makedirs('log', exist_ok = True)
csv_logger = CSVLogger('./log/training.log')


In [62]:
# from google.colab import drive
# drive.mount('/content/drive')

In [63]:
# Train
history = model.fit(
    train_data.repeat(), 
    epochs=EPOCHS,
    steps_per_epoch = 50,
    validation_data=test_data,
    validation_steps = 2,
    callbacks=[
                # tf.keras.callbacks.EarlyStopping(patience=5),
                model_ckpt,
                csv_logger]
                )

Epoch 1/10


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


50/50 [==============================] - 100s 1s/step - loss: 7.0206 - masked_acc: 0.1077 - masked_loss: 7.0206 - val_loss: 6.1559 - val_masked_acc: 0.1353 - val_masked_loss: 6.1559
Epoch 2/10
50/50 [==============================] - 60s 1s/step - loss: 5.9424 - masked_acc: 0.1339 - masked_loss: 5.9424 - val_loss: 6.0000 - val_masked_acc: 0.1434 - val_masked_loss: 6.0000
Epoch 3/10
50/50 [==============================] - 59s 1s/step - loss: 5.9743 - masked_acc: 0.1355 - masked_loss: 5.9743 - val_loss: 5.8349 - val_masked_acc: 0.1153 - val_masked_loss: 5.8349
Epoch 4/10
50/50 [==============================] - 59s 1s/step - loss: 5.7145 - masked_acc: 0.1597 - masked_loss: 5.7145 - val_loss: 5.7373 - val_masked_acc: 0.1838 - val_masked_loss: 5.7373
Epoch 5/10
50/50 [==============================] - 58s 1s/step - loss: 5.7315 - masked_acc: 0.1804 - masked_loss: 5.7315 - val_loss: 5.6236 - val_masked_acc: 0.1992 - val_masked_loss: 5.6236
Epoch 6/10
50/50 [==============================] 

In [64]:
# # Loading model checkpoint
# MODEL_CHECKPOINT_DIR = '/content/drive/MyDrive/tf_model/03-01-2023_07-07-48_epoch-30_loss-0.4649_acc-0.3174'

# model.load_weights(MODEL_CHECKPOINT_DIR)


In [65]:
model_loss = min(history.history['masked_loss'])
model_accuracy = min(history.history['masked_acc']) 
print(f'{model_loss=} \n {model_accuracy=}')

model_loss=4.958247184753418 
 model_accuracy=0.10766515135765076


In [76]:
# # saving a model as h5

# now = datetime.now().strftime('%m-%d-%Y_%H-%M-%S')
# model_name = 'chatbot_with_attention-'+now+f'_epoch-{EPOCHS}'+f'_loss-{model_loss:.4f}'+f'_acc-{model_accuracy:.4f}'

# model.save_weights(f'./saved_model/{model_name}')

# model.save(f'./saved_model/{model_name}_full', save_format='tf')


# Translate

In [91]:
@ChatBot.add_method
def reply(self,
              texts, *,
              max_length=50,
              temperature=0.0):
    # Process the input texts
    context = self.encoder.convert_input(texts, return_state = True)

    context, state = context

    batch_size = tf.shape(texts)[0]

    # Setup the loop inputs
    tokens = []
    # attention_weights = []
    next_token, done, state_zero = self.decoder.get_initial_state(context)
    # state = state
    state =[state_zero,state_zero]
    for _ in range(max_length):
        # Generate the next token
        next_token, done, state = self.decoder.get_next_token(
                next_token, context, done,  state, temperature)

        # Collect the generated tokens
        tokens.append(next_token)
        # attention_weights.append(self.decoder.last_attention_weights)

        if tf.executing_eagerly() and tf.reduce_all(done):
            break

    # Stack the lists of tokens and attention weights.
    tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
    # self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

    result = self.decoder.tokens_to_text(tokens)
    return result

In [92]:
for  i in range(0, 13):

    result = model.reply(['how was your date with john ?'], temperature = i/10)
    print(f'{i:3} : {result.numpy()[0].decode()}')

  0 : how ? 
  1 : how ? 
  2 : how ? 
  3 : how do you have a . 
  4 : how ? 
  5 : how ? 
  6 : how . you have to say . 
  7 : how to have so ? 
  8 : come are take when lord . 
  9 : why was view . 
 10 : come to you else . 
 11 : how me williams will you should president me on a could put to see percent a the minutelets wife gainfully filer kidding for meeting ship extraordinary compared my name love ? 
 12 : how do be organization thousand principle signed any end and brian . 


In [93]:
import pandas as pd
data = pd.read_csv('./dataset.csv', sep = '\t', encoding='latin1', names = ['col1','col2']) 
data.shape

(221282, 2)

In [94]:
test_num = 10
for i in range(test_num):
    a = data.sample(1)
    print('Input     : ',a['col1'].values[0])
    print('Output    : ',a['col2'].values[0])
    result = model.reply(a['col1'].values, temperature = .6)
    print('Predicted : ',    result.numpy()[0].decode())
    print()

Input     :  Not enough.  We're out numbered and out gunned.  They'd cut us to shreds.
Output    :  So good bye, Shi'ites.  Thanks for playing.  We have some lovely parting gifts for you as you go.
Predicted :  not not . 

Input     :  ...Still, I must say. I haven't felt peace like this since the grand productive days. Don't you find it so, Barton? Ain't writin' peace?
Output    :  Well... actually, no Bill...
Predicted :  who as would i think you . i didnt know to . 

Input     :  You know what I mean when I say "it."
Output    :  Gotcha. No need to get anatomically correct with me, Rex.
Predicted :  you have you have you ? 

Input     :  His hatred of the Federation is apparent. He would have built a weapon of that scope for one reason. He is going after Earth.
Output    :  Oh boy. Destroy humanity and the Federation is crippled.
Predicted :  she was , she was . . . the . 

Input     :  No.
Output    :  Yes, I mean, who wants to read about another dead priest? It's about some other 

In [95]:
!ls

 components			    model_checkpoint
'cornell movie-dialogs corpus'	    movie_dialogue_chatbot.ipynb
 cornell_movie_dialogs_corpus.zip   preprocessing_cornelldata.py
 dataset			    preprocess.py
 dataset.csv			    README.md
 embedding			    saved_model
 funcyou			    TextFile.txt
 LICENSE			    training.ipynb
 log


In [99]:
class Export(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
    def reply(self, inputs):
        return self.model.reply(inputs)

In [100]:
inputs = [
    "It's really cold here.",
    "This is my life.",
    "His room is a mess"
]

In [101]:
export = Export(model)

_ = export.reply(tf.constant(inputs))


In [103]:
result = export.reply(tf.constant(inputs))

print(result[0].numpy().decode())
print(result[1].numpy().decode())
print(result[2].numpy().decode())
print()

its .                                                
this is . . .                                             
he was .                                               



In [104]:
tf.saved_model.save(export, 'chatbot',
                    signatures={'serving_default': export.reply})

In [105]:
!pwd

/content/chatbot-with-attention/chatbot-with-attention


In [ ]:
%%time
reloaded = tf.saved_model.load('chatbot')
_ = reloaded.reply(tf.constant(inputs)) #warmup

In [ ]:
# from google.colab import runtime
# runtime.unassign()